<a href="https://colab.research.google.com/github/engineerjkk/DeepLearning_from_Scratch/blob/main/LeNet_Error1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pwd

'/content'

In [5]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Flatten, Dense
from tensorflow.keras.losses import CategoricalCrossentropy
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline

In [6]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_val, y_val = X_train[55000:, ..., np.newaxis], y_train[55000:]
X_train, y_train = X_train[:55000, ..., np.newaxis], y_train[:55000]
X_test =  X_test[..., np.newaxis]

print("Image Shape: {}".format(X_train[0].shape), end = '\n\n')
print("Training Set:   {} samples".format(len(X_train)))
print("Validation Set:   {} samples".format(len(X_val)))
print("Test Set:       {} samples".format(len(X_test)))

11501568/11490434 [==============================] - 0s 0us/step
Image Shape: (28, 28, 1)

Training Set:   55000 samples
Validation Set:   5000 samples
Test Set:       10000 samples


In [7]:
# Pad images with 0s
X_train = np.pad(X_train, ((0,0),(2,2),(2,2),(0,0)), 'constant')
X_val = np.pad(X_val, ((0,0),(2,2),(2,2),(0,0)), 'constant')
X_test = np.pad(X_test, ((0,0),(2,2),(2,2),(0,0)), 'constant')

print("Updated Image Shape for: ", end='\n\n')
print("-Training set: {}".format(X_train.shape))
print("-Validation set: {}".format(X_val.shape))
print("-Test set: {}".format(X_test.shape))

Updated Image Shape for: 

-Training set: (55000, 32, 32, 1)
-Validation set: (5000, 32, 32, 1)
-Test set: (10000, 32, 32, 1)


In [8]:
# Normalization.
X_train, X_val, X_test = X_train/float(255), X_val/float(255), X_test/float(255)
X_train -= np.mean(X_train)
X_val -= np.mean(X_val)
X_test -= np.mean(X_test)

In [9]:
def LeNet_5():
    model = Sequential()

    # C1: (None,32,32,1) -> (None,28,28,6).
    model.add(Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='tanh', input_shape=(32,32,1), padding='valid'))

    # P1: (None,28,28,6) -> (None,14,14,6).
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

    # C2: (None,14,14,6) -> (None,10,10,16).
    model.add(Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid'))

    # P2: (None,10,10,16) -> (None,5,5,16).
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

    # Flatten: (None,5,5,16) -> (None, 400).
    model.add(Flatten())

    # FC1: (None, 400) -> (None,120).
    model.add(Dense(120, activation='tanh'))

    # FC2: (None,120) -> (None,84).
    model.add(Dense(84, activation='tanh'))

    # FC3: (None,84) -> (None,10).
    model.add(Dense(10, activation='softmax'))

    # Compile the model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['accuracy']) 
    
    return model

In [11]:
model=LeNet_5()
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20)
# Save the model.
model.save("lenet5_model.h5")

Epoch 1/20
1719/1719 [==============================] - 38s 5ms/step - loss: 0.6340 - accuracy: 0.8281 - val_loss: 0.2490 - val_accuracy: 0.9336
Epoch 2/20
1719/1719 [==============================] - 8s 5ms/step - loss: 0.2705 - accuracy: 0.9213 - val_loss: 0.1785 - val_accuracy: 0.9524
Epoch 3/20
1719/1719 [==============================] - 8s 5ms/step - loss: 0.2047 - accuracy: 0.9388 - val_loss: 0.1397 - val_accuracy: 0.9612
Epoch 4/20
1719/1719 [==============================] - 8s 5ms/step - loss: 0.1619 - accuracy: 0.9519 - val_loss: 0.1133 - val_accuracy: 0.9690
Epoch 5/20
1719/1719 [==============================] - 8s 5ms/step - loss: 0.1320 - accuracy: 0.9612 - val_loss: 0.0969 - val_accuracy: 0.9716
Epoch 6/20
1719/1719 [==============================] - 8s 5ms/step - loss: 0.1114 - accuracy: 0.9679 - val_loss: 0.0889 - val_accuracy: 0.9754
Epoch 7/20
1719/1719 [==============================] - 8s 5ms/step - loss: 0.0961 - accuracy: 0.9724 - val_loss: 0.0789 - val_accuracy

In [18]:
# Restore the model.
model = tf.keras.models.load_model('lenet5_model.h5')

# Make prediction.
predictions = model.predict(X_test)

# Retrieve predictions indexes.
y_pred = np.argmax(predictions, axis=1)

# Print test set accuracy.
print('Test set error rate: {}'.format(np.mean(y_pred == y_test)))

# Plot some examples with model predictions.
print('\nSome correct classification:')
plot_example(X_test, y_test, y_pred)

print('\nSome incorrect classification:')
plot_example_errors(X_test, y_test, y_pred)

# Plot training error.
print('\nPlot of training error over 20 epochs:')
plt.title('Training error')
plt.ylabel('Cost')
plt.xlabel('epoch')

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['train loss', 'val loss'], loc='upper right')
plt.show()

Test set error rate: 0.9855

Some correct classification:


NameError: ignored

In [19]:
X_test

array([[[[-0.1014565],
         [-0.1014565],
         [-0.1014565],
         ...,
         [-0.1014565],
         [-0.1014565],
         [-0.1014565]],

        [[-0.1014565],
         [-0.1014565],
         [-0.1014565],
         ...,
         [-0.1014565],
         [-0.1014565],
         [-0.1014565]],

        [[-0.1014565],
         [-0.1014565],
         [-0.1014565],
         ...,
         [-0.1014565],
         [-0.1014565],
         [-0.1014565]],

        ...,

        [[-0.1014565],
         [-0.1014565],
         [-0.1014565],
         ...,
         [-0.1014565],
         [-0.1014565],
         [-0.1014565]],

        [[-0.1014565],
         [-0.1014565],
         [-0.1014565],
         ...,
         [-0.1014565],
         [-0.1014565],
         [-0.1014565]],

        [[-0.1014565],
         [-0.1014565],
         [-0.1014565],
         ...,
         [-0.1014565],
         [-0.1014565],
         [-0.1014565]]],


       [[[-0.1014565],
         [-0.1014565],
         [-0.1

In [20]:
y_test

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)